In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as iter_t
import copy as copy

retail_data = [i.strip().split() for i in open("retail.dat").readlines()]

retail_data = pd.DataFrame(retail_data)

# allocate a limited number of rows, for testing
retail_data = retail_data[:100]

# Random Sampling

In [138]:
def random_sampling(data, min_support=0.05, min_confidence=0.10):
    
    total_transactions = len(data)
    
    reduction = 5
    
    data = data.sample(n=int(total_transactions/reduction))
    data.reset_index(inplace=True, drop=True)
    min_support = min_support / reduction
    
    def n_candidate(prev_L, n):
        names_helper = np.array([], dtype=int)
        for i in range(n-1):
            names_helper = np.append(names_helper, prev_L[i].to_numpy(dtype=int))
            
        candidate         = iter_t.combinations(set(names_helper), n)
        candidate         = pd.DataFrame([i for i in candidate])
        if len(candidate) == 0:
            return candidate
        candidate["freq"] = np.zeros(len(candidate[0]), dtype=int)
        
        candidate.is_copy = False
        
        for i in range(data.shape[0]):
            for j in range(candidate.shape[0]):
                if set(candidate.loc[j][0:n].to_numpy(dtype=int)).issubset(set(data.loc[i].dropna().to_numpy(dtype=int))):
                    candidate.loc[j, "freq"] += 1
        return candidate
    
    # get unique items
    # construct array of all items
    items     = []
    hash_dict = {}
    for i in range(data.shape[1]):
        for j in range(data.shape[0]):
            items.append(data[i][j])
        
    unique_items = list(set(items))
    
    # construct candidate sets
    C = []
    L = []
    
    C1 = []
    for val in unique_items:
        C1.append((val, items.count(val)))
    
    #total_transactions = len(data)
    
    del(items)
    
    C1 = pd.DataFrame(C1, columns=[0, "freq"], dtype=int)
    C1 = C1.dropna()
    
    C1["conf"] = np.ones(len(C1[0]), dtype=float)
    C1["sup"]  = C1["freq"] / total_transactions
    
    L1 = C1[C1["sup"] >= min_support]
    L1 = L1.astype({0: int})
    
    C.append(C1)
    L.append(L1)
    
    def conf(_L, prevL, n):
        # Build confidence
        pd.set_option('mode.chained_assignment', 'warn')
        _L["conf"] = np.zeros(len(_L[0]), dtype=float)

        _L.is_copy = False
        for i in range(len(_L["freq"])):
            oldSup = _L["freq"].iloc[i]                                        #(prevL["freq"][prevL.iloc[:, 0:n] == L_cur[0:n]]).iat[0]
            for j in range(len(prevL[0])):
                if set(prevL.iloc[j, 0:n-1]) == set(_L.iloc[i, 0:n-1]):
                    oldSup = prevL["freq"].iloc[j]
                    break
            _L["conf"].iloc[i] = (_L["freq"].iloc[i] / oldSup)
        
        return _L
        
    
    i = 2
    
    if len(L1[0]) == 0:
        return L
    
    while True:
        cand         = n_candidate(L[i-2], i)
        if len(cand) == 0:
            break
        cand["sup"]  = cand["freq"] / total_transactions
        L_cur        = cand[cand["sup"] >= min_support]
        
        if len(L_cur[0]) == 0:
            break
        
        L_cur = conf(L_cur, L[i-2], i)
        
        L_cur = L_cur[L_cur["conf"] >= min_confidence]
        
        C.append(cand)
        L.append(L_cur)
        i += 1
    
    return L

In [139]:
res = random_sampling(retail_data, min_support=0.15)
res

/home/oboutte/.local/lib/python2.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[      0  freq  conf   sup
 75   39    10   1.0  0.10
 76   38     4   1.0  0.04
 115  48     7   1.0  0.07
 117  41     5   1.0  0.05,     0   1  freq   sup  conf
 5  38  39     4  0.04   1.0]

# SON

In [4]:
def apriori(data, min_support=0.05, min_confidence=0.10):
    def n_candidate(prev_L, n):
        names_helper = np.array([], dtype=int)
        for i in range(n-1):
            names_helper = np.append(names_helper, prev_L[i].to_numpy(dtype=int))

        candidate         = iter_t.combinations(set(names_helper), n)
        candidate         = pd.DataFrame([i for i in candidate])
        candidate["freq"] = np.zeros(len(candidate[0]), dtype=int)

        candidate.is_copy = False

        for i in range(data.shape[0]):
            for j in range(candidate.shape[0]):
                if set(candidate.loc[j][0:n].to_numpy(dtype=int)).issubset(set(data.loc[i].dropna().to_numpy(dtype=int))):
                    candidate.loc[j, "freq"] += 1
        return candidate
    
    # get unique items
    # construct array of all items
    items = []
    for i in range(data.shape[1]):
        for j in range(data.shape[0]):
            items.append(data[i][j])
    unique_items = list(set(items))
    
    # construct candidate sets
    C = []
    L = []
    
    C1 = []
    for val in unique_items:
        C1.append((val, items.count(val)))
    
    del(items)
    
    C1 = pd.DataFrame(C1, columns=[0, "freq"], dtype=int)
    C1 = C1.dropna()
    
    C1["conf"] = np.ones(len(C1[0]), dtype=float)
    
    L1 = C1[C1["freq"] >= min_support]
    L1 = L1.astype({0: int})
    
    C.append(C1)
    L.append(L1)
    
    def conf(_L, prevL, n):
        # Build confidence
        pd.set_option('mode.chained_assignment', 'warn')
        _L["conf"] = np.zeros(len(_L[0]), dtype=float)

        _L.is_copy = False
        for i in range(len(_L["freq"])):
            oldSup = _L["freq"].iloc[i]                                        #(prevL["freq"][prevL.iloc[:, 0:n] == L_cur[0:n]]).iat[0]
            for j in range(len(prevL[0])):
                if set(prevL.iloc[j, 0:n-1]) == set(_L.iloc[i, 0:n-1]):
                    oldSup = prevL["freq"].iloc[j]
                    break
            _L["conf"].iloc[i] = (_L["freq"].iloc[i] / oldSup)
        
        return _L
        
    
    i = 2
    while True:
        cand  = n_candidate(L[i-2], i)
        L_cur = cand[cand["freq"] >= min_support]
        
        if len(L_cur[0]) == 0:
            break
        
        L_cur = conf(L_cur, L[i-2], i)
        
        L_cur = L_cur[L_cur["conf"] >= min_confidence]
        
        C.append(cand)
        L.append(L_cur)
        i += 1
    
    return L



def apriori_SON(data, chunks=2, min_support=0.15, min_confidence=0.10):
    
    #calcuclate local support amount based on s*n/k formula
    chunk_size=data.shape[0]/chunks
    ls = min_support*chunks
    
    #PASS 1
    #split dataset into chunks
    chunk_list = np.array_split(data, chunks)

    #apply apriori to each chunk to get the local candidate sets (stored in results)
    results = []
    for chunk in chunk_list:
        chunk.reset_index(inplace=True, drop=True)
        results.append(apriori(chunk, min_support=ls, min_confidence=min_confidence))
    
    #PASS 2
    #sort each candidate set
    for i in range(0, chunks):
        for candidate in results[i]:
            col_list = []
            for x in range(candidate.shape[1]-2):
                col_list.append(x)
            candidate = candidate.sort_values(by=col_list, ascending=True)
            
    #create dictionary for counts
    d = {}
    for i in range(0, chunks):
        for candidate in results[i]:
            col_list = []
            for x in range(candidate.shape[1]-2):
                col_list.append(x)
            for cand in (candidate.loc[:,col_list]).to_numpy():
                if tuple(cand) not in d:
                    d[tuple(cand)] = 0
    
    #for each chunk, go through and count how many times each candidate set appears 
    for chunk in results:
        for cand in chunk:
            n = cand.shape[1]-2
            cand.reset_index(inplace=True, drop=True) # need to be able to properly index
            for i in range(data.shape[0]):
                    for j in range(cand.shape[0]):
                        if set(cand.loc[j][0:n].to_numpy(dtype=int)).issubset(set(data.loc[i].dropna().to_numpy(dtype=int))):
                            d[tuple(cand.loc[j][0:n].to_numpy())] += 1
                    
    #remove those with support lower than threshold   
    return dict([(key,value) for key, value in d.items() if (value/data.shape[0]) >= min_support])

In [ ]:
res = apriori_SON(retail_data, chunks=2, min_support=0.40)
#res = apriori(retail_data, min_support=0.15)
print(res)


/home/oboutte/.local/lib/python2.7/site-packages/pandas/core/generic.py:5079: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/home/oboutte/.local/lib/python2.7/site-packages/pandas/core/generic.py:5080: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)
